# TensorFlow

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
import tensorflow as tf
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import sys

from tqdm import tqdm

print(tf.__version__)

%store -r __wideboy

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.9.0


In [26]:
__wideboy

# A General Similarity
### TensorFlow

In [25]:
a = tf.constant([1,2,3])
b = tf.constant([4,5,6])
out1 = (a+a)*b
out2 = tf.tensordot(a, b, 1)

sess = tf.Session()
print(sess.run(out1))
print(sess.run(out2))

[ 8 20 36]
32


# A More Complex Model
### TensorFlow CNN

![title](modelClean.png)

# Data Loaders
### TensorFlow

In [11]:
def imageMod(x,size):
    return np.array(Image.fromarray(np.reshape(x,[28,28])).resize([size,size]))/255

def load_data(in_size):
    mnist = tf.keras.datasets.mnist
    (x_train, y_train), (x_test, y_test) = mnist.load_data()

    #rescale test images to 16x16
    x_train  = np.array([imageMod(x,in_size) for x in x_train]).astype(np.float32)
    x_test   = np.array([imageMod(x,in_size) for x in x_test]).astype(np.float32)

    y_train = y_train.astype(np.int32)
    y_test  = y_test.astype(np.int32)
    
    return (x_train, y_train), (x_test, y_test)

# Model Definitions
### TensorFlow

In [12]:
def cnn_model(features, labels, mode):
    
    # Input Layer
    input_layer = tf.reshape(features["x"], [-1, 16, 16, 1])

    #CONV 1
    conv1 = tf.layers.conv2d(
        inputs=input_layer,
        filters=16,
        kernel_size=[3, 3],
        padding="same")
    batchNorm1 = tf.layers.batch_normalization(conv1)
    relu1 = tf.nn.relu(batchNorm1)
    pool1 = tf.layers.max_pooling2d(inputs=relu1, pool_size=[4, 4], strides=1)

    #CONV 2
    conv2 = tf.layers.conv2d(
        inputs=pool1,
        filters=32,
        kernel_size=[3, 3],
        padding="same")
    batchNorm2 = tf.layers.batch_normalization(conv2)
    relu2 = tf.nn.relu(batchNorm2)
    pool2 = tf.layers.max_pooling2d(inputs=relu2, pool_size=[4, 4], strides=1)

    #RESHAPE
    flatten = tf.layers.flatten(pool2)
    
    #FC1
    dropout = tf.layers.dropout(inputs=flatten, rate=0.5, training=mode == tf.estimator.ModeKeys.TRAIN)
    dense = tf.layers.dense(inputs=dropout, units=32, activation=tf.nn.relu)
    
    #OUTPUT
    logits = tf.layers.dense(inputs=dropout, units=10)
    predictions = {
      "classes": tf.argmax(input=logits, axis=1),
      "probabilities": tf.nn.softmax(logits, name="softmax_tensor")
    }

    #define test functionality
    if mode == tf.estimator.ModeKeys.PREDICT:
        return tf.estimator.EstimatorSpec(mode=mode, predictions=predictions)

    #Calculate loss
    loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

    #define train functionality
    if mode == tf.estimator.ModeKeys.TRAIN:
        optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.05)
        train_op = optimizer.minimize(loss=loss,global_step=tf.train.get_global_step())
    
        return tf.estimator.EstimatorSpec(mode=mode, loss=loss, train_op=train_op)

    # Add evaluation metrics (for EVAL mode)
    eval_metric_ops = {"accuracy": tf.metrics.accuracy(labels=labels, predictions=predictions["classes"])}
    
    return tf.estimator.EstimatorSpec(mode=mode, loss=loss, eval_metric_ops=eval_metric_ops)

# Set Parameters
### TensorFlow

In [13]:
epochs = 10
lr = .05
batch_size = 200
in_size = 16

# Train Loop
### TensorFlow

In [14]:
(x_train, y_train), (x_test, y_test) = load_data(in_size)

print("Train Shape:",x_train.shape,y_train.shape)
print("Test Shape:",x_test.shape,y_test.shape)

mnist_classifier = tf.estimator.Estimator(
    model_fn=cnn_model, model_dir="/tmp/mnist_convnet_model")

# Train the model
train_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_train},
    y=y_train,
    batch_size=batch_size,
    num_epochs=None,
    shuffle=True)

mnist_classifier.train(
    input_fn=train_input_fn,
    steps=batch_size*10)

Train Shape: (60000, 16, 16) (60000,)
Test Shape: (10000, 16, 16) (10000,)
INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': '/tmp/mnist_convnet_model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x12642f5f8>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INF

KeyboardInterrupt: 

# Evaluate
### TensorFlow

In [ ]:
eval_input_fn = tf.estimator.inputs.numpy_input_fn(
    x={"x": x_test},
    y=y_test,
    num_epochs=1,
    shuffle=False)

eval_results = mnist_classifier.evaluate(input_fn=eval_input_fn)
print(eval_results)